# Bayesian Forecaster

#### Imports

In [5]:
import numpy as np
import pymc as pm
import pytensor.tensor as pt
import pandas as pd
import arviz as az

#### data setup

In [6]:
# importing data
data_directory = '../data/experiment_small/'
data = np.load(data_directory + 'network_params/data_network.npy')
data = np.transpose(data, (2,1, 0))

#reshaping and trimming to exclude the last year
data = data.reshape(len(data),2*len(data[0]))
data = data[:-3*365,:]
T,D = data.shape

# creating the dataframe
hospital_list = np.loadtxt(data_directory + 'network_params/hospitals.txt', dtype=str, delimiter='\n')
df_cols = np.array([[h + ' supply',h + ' demand'] for h in hospital_list]).reshape(D)
df = pd.DataFrame(data, columns=df_cols)

#### constructing the BVAR model

In [10]:
def ar_update(beta,n,h,df):
    update = []
    for i in range(n):
        beta_temp = pm.math.sum([
            pm.math.sum(beta[i,j] * df.values[h-(j+1):-(j+1)],axis=-1)
            for j in range(h)
        ],axis=0)
        
        update.append(beta_temp)

    update = pm.math.stack(update, axis=-1)
    return update

def bvar_model(h,df,priors):
    coords = {
        "lags": np.arange(h)+1,
        "vars": df.columns.tolist(),
        "time": [t for t in df.index[h:]]
    }

    with pm.Model(coords=coords) as model:
        beta = pm.Normal(
            "beta",
            mu=priors["beta"]["mu"],
            sigma=priors["beta"]["sigma"],
            dims=("vars","lags","vars")
        )
        alpha = pm.Normal(
            "alpha",
            mu=priors["alpha"]["mu"],
            sigma=priors["alpha"]["sigma"],
            dims=("vars",)
        )
        data_obs = pm.Data(
            "data_obs",
            df.values[h:], 
            dims=("time","vars"),
            mutable=True
        )

        n = df.shape[1]
        betaX = ar_update(beta,n,h,df)
        betaX = pm.Deterministic(
            "betaX",
            betaX,
            dims=("time",)
        )
        mu = alpha + betaX
        sigma = pm.HalfNormal(
            "noise",
            sigma=priors["noise"]["sigma"],
            dims=["vars"]
        )
        obs = pm.Normal(
            "obs",
            mu=mu,
            sigma=sigma,
            observed=data_obs,
            dims=["time", "vars"]
            )
        trace = pm.sample(chains=4, random_seed=42)
    return model,trace

#### results

In [ ]:
h = 3
priors = {
    "beta": {"mu": 0.0, "sigma": 100},
    "alpha": {"mu": 0, "sigma": 100},
    "noise": {"sigma": 10}
}

model, idata = bvar_model(h, df, priors)

Sampling: [alpha, beta, noise, obs]
Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (4 chains in 4 jobs)
NUTS: [beta, alpha, noise]


In [ ]:
az.summary(idata, var_names=["alpha", "beta","noise"])
az.plot_trace(idata)